### Creating SQLite

In [ ]:
import sqlite3

# create and connect to database
conn = sqlite3.connect("instagram_artists_apresearch.db")
cursor = conn.cursor()

cmd = ""

In [ ]:
# create table of all profiles
# column is_human is true if profile is a human artist
cmd = """CREATE TABLE profiles(
    user_id INT,
    username VARCHAR(30),
    num_followers INT,
    num_posts INT,
    is_human BOOLEAN,
    PRIMARY KEY (user_id)
    );"""
cursor.execute(cmd)

# create table of all posts
# column is_reel is true if the post is a reel
cmd = """CREATE TABLE posts(
owner_user_id INT,
owner_username VARCHAR(30),
num_likes INT,
num_comments INT,
shortcode VARCHAR(255),
img_url VARCHAR(255),
post_date DATE,
is_reel BOOLEAN,
FOREIGN KEY(owner_user_id) REFERENCES Profiles(user_id)
);"""
cursor.execute(cmd)

### Creating the dataframe of artist profiles

In [ ]:
# %pip install instaloader
import instaloader

# Creating an instance of the Instaloader class
bot = instaloader.Instaloader()
# #bot.login(user="Your_username",passwd="Your_password") #Use this code to log-in to your account.

In [ ]:
# %pip install pandas
import pandas as pd

# creating a data frame for human artists usernames
df = pd.read_csv("HA.csv")

# for every profile in the csv, retreive data from instagram using instaloader
# insert every profile as a row into the profiles table
for i in df.index:
    for col in df.columns[1:]:
        try:
            if(pd.notna(df.loc[i,col])):
                p = instaloader.Profile.from_username(bot.context, df.loc[i,col]) # retreive profile
                print(f"{p.userid}, {p.username}, {p.followers}, {p.mediacount}, true, {df.iloc[i,0]}")
        except Exception as e:
            print(f"{e} with user: {df.loc[i,col]}, row: {i}, col: {col}")

In [ ]:
# creating a data frame for AI artists usernames
df = pd.read_csv("AI.csv")

# for every profile in the csv, retreive data from instagram using instaloader
# insert every profile as a row into the profiles table
for i in df.index:
    for col in df.columns[1:]:
        try:
            if(pd.notna(df.loc[i,col])):
                p = instaloader.Profile.from_username(bot.context, df.loc[i,col]) # retreive profile
                print(f"{p.userid}, {p.username}, {p.followers}, {p.mediacount}, false, {df.iloc[i,0]}")
        except Exception as e:
            print(f"{e} with user: {df.loc[i,col]}, row: {i}, col: {col}")

In [ ]:
# create pd dataframe of all artist profiles from database
profiles_df = pd.read_sql("SELECT * FROM profiles", conn)

### Creating the dataframe of all posts from every artist profile

In [ ]:
for user in profiles_df['username']:
    profile = instaloader.Profile.from_username(bot.context, user)
    for p in profile.get_posts():
        print(f"{p.owner_id}, {p.owner_username}, {p.likes}, {p.comments}, {p.shortcode},{p.owner_id}")
